# Numpy LDPC BP Decoder 

Test numpy implementation of the decoder against Sionna reference.

In [7]:
import numpy as np
import tensorflow as tf
from sionna.phy.fec.ldpc import LDPC5GEncoder, LDPC5GDecoder

from numpy_decoder import *

In [6]:
############################
# Input Parameters
############################
BG = 2 # baseghraph selector; 1 or 2
Z = 4 # lifting factor; between 2 and 384
block_length = 40 # number of returned bits
num_iter = 2 # number of decoding iterations


bg_vn, bg_cn, bg_vn_degree, bg_cn_degree, num_cols, num_rows, num_edges = init_basegraph(BG, Z)

# input llrs to the decoder (INPUT parameter)
num_vns = num_cols * Z
llr_ch = np.ones((num_vns,))
llr_ch[:2*Z] = 0

bits, llr_out = decode_ldpc(BG, Z, block_length, num_iter, llr_ch)

print(f"llr_ch: \n{llr_ch} \n")
print(f"llr_out: \n{llr_out} \n")


llr_ch: 
[0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] 

llr_out: 
[22 22 22 22 23 23 23 23 11 11 11 11  6  6  6  6  6  6  6  6 15 15 15 15
  8  8  8  8 14 14 14 14  7  7  7  7  9  9  9  9 10 10 10 10 17 17 17 17
 10 10 10 10 13 13 13 13  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  3  3  3  3  2  2  2  2  2  2  2  2  3  3  3  3  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  

## Testing against Sionna Reference

In [16]:
# Parameters for the test
num_iter = 4
llr_mag = 10
k = 30
n = 46
Zc = 5
BG = 2
num_vn = 52*Zc
parity_start = 10*Zc

# initialize sionna reference encoder and decoder
enc = LDPC5GEncoder(k, n)
dec = LDPC5GDecoder(enc, hard_out=False, num_iter=num_iter, cn_update='minsum',
                    return_infobits=True, prune_pcm=False)
dec._llr_max=1000 # deactivate clipping

u = np.random.randint(0,2,k)
c = enc(np.expand_dims(u.astype(np.float64),0))[0].numpy()
x = 1-2*c
llr_ch = np.clip((llr_mag * x),-127,127)
#llr_ch = np.abs(llr_ch)
llr_input = np.zeros(num_vn)
llr_input[2*Zc:k] = llr_ch[:k-2*Zc] # unpunctured message bits
llr_input[k:parity_start] = 127      # shortened bits
llr_input[parity_start:parity_start+n-k+2*Zc] = llr_ch[k-2*Zc:] # parity bits

# numpy implementation
DAMPING_FACTOR = 1. # heuristic value 0.75 is typically a good
decoder_outputs, llr_output = decode_ldpc(BG, Zc, k, num_iter, llr_input)

# Run Sionna reference decoder
# Sionna uses a different sign convention for the LLRs
llr_ref = -1.* dec(-1*tf.expand_dims(tf.cast(llr_ch, tf.float32),0))[0].numpy()

# unpack bits as the decoder returns a byte array
u_hat = np.unpackbits(decoder_outputs.astype(np.uint8))[:k] # there may be a couple of extra bits if k is not a multiple of 8

# check if the decoded message matches the original message
if np.all(u == u_hat):
    print("Test passed!\n")
else:
    print("Test failed!\n")

print(f"llr_input: \n{llr_input} \n")

# return only the k first LLRs
print(f"llr_output: \n{llr_output[:k]} \n")
print(f"llr_ref: \n{llr_ref} \n")


Test passed!

llr_input: 
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  10. -10.  10. -10.
 -10.  10.  10.  10. -10.  10.  10. -10. -10.  10. -10. -10.  10.  10.
  10.  10. 127. 127. 127. 127. 127. 127. 127. 127. 127. 127. 127. 127.
 127. 127. 127. 127. 127. 127. 127. 127.  10. -10.  10.  10. -10.  10.
  10. -10. -10.  10. -10.  10.  10. -10.  10. -10.  10. -10. -10. -10.
 -10.  10.  10. -10.  10.  10.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 

## Unittest

Test that decoder can reconstruct payload for many different 5G code parameters.

In [17]:
import numpy as np
from sionna.phy.fec.ldpc.encoding import LDPC5GEncoder

num_iter = 8
llr_mag = 10

Z_vals = np.array([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                   15, 16, 18, 20, 22, 24, 26, 28, 30, 32, 36, 40, 44,
                   48, 52, 56, 60, 64, 72, 80, 88, 96, 104, 112, 120, 128,
                   144, 160, 176, 192, 208, 224, 240, 256, 288, 320, 352, 384])

kb = np.array([10,22])

k_vals = np.sort(np.outer(Z_vals, kb).flatten())
rates =  [.2,.34,.5,.66,.75,.88]

for k in k_vals:
    for r in rates:
        n = int(np.ceil(k/r))
        r_actual = k / n
        if k <= 292 or (k <= 3824 and r_actual <= 2/3) or r_actual <= 1/4:
            BG = 2
        else:
            BG = 1
        # exclude the cases that are not supported in the standard
        if BG == 1 and (r_actual > 11/12 or r_actual < 1/3): continue
        if BG == 2 and (r_actual > 10/12 or r_actual < 1/5 or k >= 3824): continue

        enc = LDPC5GEncoder(k, n)
        BG_sionna = 1 if enc._bg == "bg1" else 2
        assert BG_sionna == BG

        u = np.random.randint(0,2,k)
        c = enc(np.expand_dims(u.astype(np.float64),0))[0].numpy()
        x = 1-2*c

        llr_ch = np.clip((llr_mag * x),-127,127).astype(np.int8)

        Zc = enc._z
        BG = 1 if enc._bg == "bg1" else 2
        num_vn = 68*Zc if BG == 1 else 52*Zc
        parity_start = 22*Zc if BG == 1 else 10*Zc

        llr_input = np.zeros(num_vn,dtype=np.int8) # takes care of the punctured bits (initialized to 0 LLR)
        llr_input[2*Zc:k] = llr_ch[:k-2*Zc] # unpunctured message bits
        llr_input[k:parity_start] = 127      # shortened bits
        llr_input[parity_start:parity_start+n-k+2*Zc] = llr_ch[k-2*Zc:] # parity bits
        llr_input = llr_input.astype(np.int32)
        decoder_outputs, llr_output = decode_ldpc(BG, Zc, k, num_iter,
                                                  llr_input)

        u_hat = np.unpackbits(decoder_outputs.astype(np.uint8))[:k] # there may be a couple of extra bits if k is not a multiple of 8
        print(u)
        print(u_hat)

        if np.all(u == u_hat):
            print(f'K = {k:4}, N = {n:5}, Rc = {k/n:.2f}, BG = {BG:1}, Z = {Zc:3}... ', end='', flush=True )
            print('Success')
        else:
            print(f'K = {k:4}, N = {n:5}, Rc = {k/n:.2f}, BG = {BG:1}, Z = {Zc:3}... ', end='', flush=True )
            print('Failed')
        #assert False


[1 1 0 1 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0]
[1 1 0 1 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0]
K =   20, N =   100, Rc = 0.20, BG = 2, Z =   4... Success
[1 0 1 0 0 1 1 1 1 1 0 1 0 0 0 0 1 1 0 0]
[1 0 1 0 0 1 1 1 1 1 0 1 0 0 0 0 1 1 0 0]
K =   20, N =    59, Rc = 0.34, BG = 2, Z =   4... Success
[1 1 0 1 1 0 0 0 1 0 0 0 1 0 1 1 1 0 0 1]
[1 1 0 1 1 0 0 0 1 0 0 0 1 0 1 1 1 0 0 1]
K =   20, N =    40, Rc = 0.50, BG = 2, Z =   4... Success
[1 1 1 0 1 1 1 0 1 0 0 1 1 0 1 1 1 1 0 1]
[1 1 1 0 1 1 1 0 1 0 0 1 1 0 1 1 1 1 0 1]
K =   20, N =    31, Rc = 0.65, BG = 2, Z =   4... Success
[1 0 1 0 0 0 0 0 1 1 1 1 0 1 0 0 0 1 0 1]
[1 0 1 0 0 0 0 0 1 1 1 1 0 1 0 0 0 1 0 1]
K =   20, N =    27, Rc = 0.74, BG = 2, Z =   4... Success
[0 1 1 1 0 0 0 1 1 0 0 1 1 1 0 0 0 0 1 1 1 0 0 1 0 0 1 1 0 1]
[0 1 1 1 0 0 0 1 1 0 0 1 1 1 0 0 0 0 1 1 1 0 0 1 0 0 1 1 0 1]
K =   30, N =   150, Rc = 0.20, BG = 2, Z =   5... Success
[0 0 1 1 1 0 0 0 1 0 0 1 1 1 1 1 0 0 1 1 0 0 0 1 1 1 1 1 1 0]
[0 0 1 1 1 0 0 0 1 0 0 1 1 1 1 1 0 0 1 1